### 방 사진 정보 크롤링

ano -> accommodationId 와 accommodationId, roomName -> roomId를 찾아서 picture  추가

데이터를 확인하면 room과 accommodation이 잘못 매핑된 경우도 있는 것 같음 (추후 수정)


In [2]:
import requests
import json
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import re

In [3]:
# 숙소상세정보 데이터 불러오기

acc_detail = pd.read_csv(f'../data/숙소상세정보.csv')
acc_detail.head()

,Unnamed: 0,id,category,name,score,region,ano,lat,lng,pic_url,address,pic_urls,detail
0,0,1,1,강남 더 뮤즈,9.8,강남구 역삼동,1824,37.502530,127.038275,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-9,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
1,1,2,1,역삼 호텔 디 아티스트,9.3,강남구 역삼동,5675,37.502415,127.038488,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 678-13,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
2,2,3,1,역삼 리치웰,9.3,강남구 역삼동,3805,37.502973,127.039823,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 677-9,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
3,3,4,1,강남 캠퍼스,9.3,강남구 역삼동,4988,37.497310,127.029190,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 825-30,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."
4,4,5,1,역삼 인트로호텔,9.9,강남구 역삼동,72748,37.502754,127.039923,https://image.goodchoice.kr/resize_1000X500x0/...,서울 강남구 역삼동 677-14,['https://image.goodchoice.kr/resize_490x348/a...,"<article class=""detail_info"">\n<!-- 기본 정보 -->\..."


In [4]:
# 방상세정보 데이터 불러오기
room_detail = pd.read_csv(f'../data/방상세정보.csv')
room_detail.head()

,Unnamed: 0,roomId,accommodationId,name,checkIn,checkOut,defaultPeople,maxPeople
0,0,1,1,랜덤객실,23.0,12,2,2
1,1,2,1,로이,23.0,12,2,2
2,2,3,1,샴페인골드,23.0,12,2,2
3,3,4,1,미스트,23.0,12,2,2
4,4,5,1,에메랄드,23.0,12,2,2


In [44]:
# 저장한 숙소 목록의 상세정보 데이터 가져오기
image_list = []

for idx in tqdm(range(len(acc_detail))):
    cur = acc_detail.iloc[idx]
    ano = cur['ano']
    id = cur['id']
    category = cur['category']

    time.sleep(0.1)
    url = f"https://www.goodchoice.kr/product/detail?ano={ano}&sel_date=2023-09-30&sel_date2=2023-10-01"

    resp = requests.get(url)
    soup = BeautifulSoup(resp.content , "html.parser")
    html = soup.select_one("#content")

    # 방 상세정보
    rooms = html.select_one('.room_info')

    for room in rooms.select('.room'):
            
        try:
            name = room.select_one('.title').text.strip()
            
            roomId = room_detail[(room_detail["accommodationId"] == id) & (room_detail["name"] == name)].iloc[0]["roomId"]

            images = room.select_one('.pic_wrap').select(".gallery_m .item")
            for image in images:
                image_list.append({
                     "url" : "https:"+image.select_one("img")["data-src"],
                     "roomId" : roomId
                })
        except:
            print("error", name, id)

 15%|█▍        | 39/266 [00:30<03:05,  1.22it/s]

error 스위트 루프탑(수영장 이용불가,글램핑,트윈,... 39
error 스위트 루프탑(수영장이용가능,글램핑,트윈,... 39
error 파티스위트 루프탑(수영장이용가능,글램핑,트... 39
error 무한대실 디럭스(욕조유무 랜덤배정, 넷플릭... 39
error 파티스위트 루프탑(수영장 이용불가,글램핑, ... 39
error 도보전용 디럭스(주차불가/랜덤배정/넷플릭스... 39
error 디럭스A(랜덤배정/넷플릭스-본인계정) 39
error 디럭스 트윈A(트윈베드,넷플릭스(본인계정)) 39
error 디럭스B(랜덤배정/넷플릭스-본인계정) 39
error 프리미엄 트윈(트윈베드,넷플릭스(본인계정)) 39
error 디럭스 트윈B(트윈베드,넷플릭스(본인계정)) 39


 15%|█▌        | 40/266 [00:31<02:56,  1.28it/s]

error STANDARD ROOM 40
error DELUXE ROOM 40
error SUITE ROOM 40
error ROYAL SUITE ROOM 40


 15%|█▌        | 41/266 [00:31<02:53,  1.30it/s]

...

 36%|███▋      | 97/266 [01:15<02:24,  1.17it/s]

error 로프트 더블 97
error 스튜디오 트윈 97
error [Sunset with wine 패키지] 로프트 더블 97
error [서울 뷰 맛집 패키지] 로프트 더블 97
error [Sunset with wine 패키지] 스튜디오 트윈 97
error [서울 뷰 맛집 패키지] 스튜디오 트윈 97


 68%|██████▊   | 180/266 [02:10<01:00,  1.41it/s]

error 2호 쥬피터 180
error 3호 헤라 180
error 4호 비너스 180
error 1호 라운지 180
error 5호 헤르메스 180


100%|██████████| 266/266 [02:59<00:00,  1.48it/s]


In [46]:
# 데이터 프레임으로 변환
room_picture = pd.DataFrame(image_list)
room_picture

# 방 사진정보 저장
room_picture.to_csv(f"../data/방사진.csv" ,encoding="utf-8")
room_picture.head(3)

,url,roomId
0,https://image.goodchoice.kr/resize_490x348/adi...,1
1,https://image.goodchoice.kr/resize_490x348/adi...,1
2,https://image.goodchoice.kr/resize_490x348/adi...,1


In [50]:
room_picture = pd.read_csv(f"../data/방사진.csv")
room_picture = room_picture.rename(columns={'Unnamed: 0' : "pictureId"})
room_picture["pictureId"] = room_picture["pictureId"] + 1

room_picture.head()

,pictureId,url,roomId
0,1,https://image.goodchoice.kr/resize_490x348/adi...,1
1,2,https://image.goodchoice.kr/resize_490x348/adi...,1
2,3,https://image.goodchoice.kr/resize_490x348/adi...,1
3,4,https://image.goodchoice.kr/resize_490x348/adi...,1
4,5,https://image.goodchoice.kr/resize_490x348/adi...,1


In [51]:
room_picture.to_csv(f"../data/방사진.csv" ,encoding="utf-8")
room_picture.head(3)

,pictureId,url,roomId
0,1,https://image.goodchoice.kr/resize_490x348/adi...,1
1,2,https://image.goodchoice.kr/resize_490x348/adi...,1
2,3,https://image.goodchoice.kr/resize_490x348/adi...,1
